In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import time
from numba import njit

from propagation import received_power, quad_distribute_power, propagation_delay, \
                        generate_aoa2_rx_signals, generate_rtof_rx_signals, generate_pdoa_rx_signals, \
                        map_rx_config, map_tx_config

from parameter_measurement import aoa_measurement, rtof_d_measure, pdoa_deld_measure

from position_estimation import aoa2_positioning, rtof_bechadergue_positioning

In [2]:
f_simulation = 1e10;           # [Hz], simulation clock freq that should capture light delays amounting to 3cm
t_simulation = 1/f_simulation; # [s] , signal simulation period

f_clock = 1e8;         # [Hz], measurement clock freq
t_clock = 1/f_clock;   # [s] , measurement clock period
f_e     = 1.00e6;      # [Hz], emitted wave freq

simulation_start  = t_simulation; # [s], t_clock rather than 0 avoids artifacts and sim ends at simulation_stop this way
simulation_stop   = 0.001;        # [s]
simulation_length = int(simulation_stop/t_simulation)

s_simulation = np.linspace(simulation_start, simulation_stop, simulation_length)

clock_subsample_rate = int(f_simulation/f_clock)

s_clock = np.zeros_like(s_simulation)
for i in range(0, int(clock_subsample_rate/2)):
    s_clock[ i::clock_subsample_rate] = 1;

s_clock_lead  = s_clock[1:]
s_clock_lag   = s_clock[0:-1]
s_clock_re    = np.concatenate((np.asarray([False]), np.logical_and((1-s_clock_lag),s_clock_lead)))
del s_clock, s_clock_lead, s_clock_lag

c = 299792458 # [m/s] speed of light

#fp = 2*np.pi*f_e*np.linspace(49, 50, 981)/c # phase shift, "from 1 m to 50 m and taking one measurement every 5 cm."
#xp = np.linspace(t_clock, simulation_stop, fp.shape[0]);

x_tx = 2
y_tx = 5
L    = 1.6

phase_shift_rxL = 2*np.pi*f_e*np.sqrt(x_tx**2+y_tx**2)/c
phase_shift_rxR = 2*np.pi*f_e*np.sqrt((x_tx-L)**2+y_tx**2)/c

In [3]:
#emitted = np.sin(2*np.pi* f_e *s_simulation - np.pi/32);
rxLL = np.sin(2*np.pi* f_e *s_simulation - phase_shift_rxL - np.pi/32);
rxRL = np.sin(2*np.pi* f_e *s_simulation - phase_shift_rxR - np.pi/32);

start = time.time()
phase_shift_diff_est_txL = pdoa_deld_measure(rxLL, rxRL)
print(time.time() - start)

print(phase_shift_rxL-phase_shift_rxR)
print(phase_shift_diff_est_txL)

1.9039192199707031
0.007737657059372446
0.00773765705951304


In [4]:
rxLL = np.fft.fft(rxLL);
rxLL[0:int(rxLL.shape[0]/2)] = 0;
rxLL = np.fft.ifft(rxLL);
rxRL = np.fft.fft(rxRL);
rxRL[0:int(rxRL.shape[0]/2)] = 0;
rxRL = np.fft.ifft(rxRL);
phase_shift_diff_est = np.mean(np.angle(rxLL * np.conjugate(rxRL)));
